# This code records the play-by-play games of GSW 2015-2016.

# The end of this code generates two csv files, the Position Dataset, and the Names Dataset. 

# The idea is to use these datasets to generate the ultimate csv file needed for multiple regression.

### Functions

In [1]:
from bs4 import BeautifulSoup
import requests
from HTMLParser import HTMLParser
from datetime import datetime
import hashlib
import requests
import re

import numpy as np
import pandas as pd

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()


# In[58]:
from selenium import webdriver
import urllib2
import MySQLdb

In [2]:
#get the difference of two times

def subtract_time(present, past):
    
    present_copy = present
    past_copy = past
    
    #if negative, then borrow from past's min
    if past_copy['sec'] - present_copy['sec'] < 0:
        
        past_copy['min'] -= 1
        past_copy['sec'] += 60
    
    min_diff = past_copy['min'] - present_copy['min']
    sec_diff = past_copy['sec'] - present_copy['sec']
    
    return({'min': min_diff, 'sec':sec_diff})


In [37]:
#a function to parse the time and update it in dict

def update_time(time_parsed, time):
    
    min_sec = time_parsed.split(":")
    mins = int(min_sec[0])
    secs = int(min_sec[1].split(".")[0])
    
    time['min'] = mins
    time['sec'] = secs
    
    #print(time)


In [38]:
#a function to update the scores onto dicts

def update_scores(score_parsed, scores):
    rel_score = score_parsed.split("]")[0]
    
    b = rel_score.split("]")[0]
    #print (b)

    team_left = b.split(" ")[0]
    scores_team = b.split(" ")[1]

    left_team_score = int(scores_team.split("-")[0])
    right_team_score = int(scores_team.split("-")[1])

    if (team_left == "GSW"):
        scores['GSW'] = left_team_score
        scores['OPP'] = right_team_score
    else:
        scores['OPP'] = left_team_score
        scores['GSW'] = right_team_score

    

In [39]:
#makes a substitution to the current 5 players on the team

def replace_players(team_to_replace, text):
    split_text = text.split(" Substitution replaced by ")
    to_bench = split_text[0].replace(u'\xa0', u'')
    to_play = split_text[1].replace(u'\xa0', u'')
    team_copy = team_to_replace[:]
    
    for player in team_to_replace:
        #print(player, to_bench)
        if to_bench[2:] in player:
            team_copy[team_copy.index(player)] = to_play
            break
    return(team_copy)
    


In [40]:
#def predict_new_starting_5(previous_5, data_parsed, NBA_dataset):
def get_player_count(list_names):
    dd = {}
    for i in set(list_names):
        dd[i] = list_names.count(i)
    
    return(dd)

# a function to predict starting 5 in a new quarter

def predict_new_starting_5(team, previous_5, data_parsed, NBA_dataset, mark):
    
    new_s5 = []
    
    players_ln = NBA_dataset[NBA_dataset['team'] == team]['last_name'].values.tolist()

    #forbidden players are those players after the Substitution replaced 
    #forbidden_players = get_forbidden_players(data_parsed, mark)
    forbidden_players = []
    for i in data_parsed[mark:]:
        
        if len(new_s5) == 5:
            return (new_s5)

        data = strip_tags(str(i)).strip(" ")
        first_word = data.split(" ")[0].strip(" ")
            
        #if starting 5 is not yet complete and someone arrived, that someone is forbidden as a player
        
        if len(new_s5) < 5 and " Substitution replaced by " in data:
            to_arrive = data.split(" Substitution replaced by ")[-1]
            forbidden_players.append(to_arrive[:-2].strip(" "))
            
            to_go = data.split(" Substitution replaced by ")[0]
            # check if the one going is a starting 5, then append it as well
            if to_go not in forbidden_players and to_go not in new_s5:
                new_s5.append(to_go)
            
            continue
            
        #normal case
        if first_word in players_ln:
            if first_word not in new_s5 and first_word not in forbidden_players:
                new_s5.append(first_word)
            continue

        
        #tricky part
        else:

            #get the second word for help
            if len(data) > 2:
                second_word = data.split(" ")[1].strip(" ")
            else:
                second_word = ''
                continue

            first_2 = first_word + " " + second_word
            for word in ['Team','Rebound','Delay','Violation','Technical', 'Jump']:
                if word in first_2:
                    continue

            
            #K Thompson case
            ln_player_count = get_player_count(players_ln)
            
            if second_word in players_ln:
                if ln_player_count[second_word] > 1:
                    if first_2 not in new_s5 and first_2 not in forbidden_players:
                        new_s5.append(first_2)
                    continue
                    
            
            
            #Mbah a Moute and McAdoo case
            
            for name in players_ln:
                if first_word in name:
                    if name not in new_s5 and name not in forbidden_players:
                        new_s5.append(name)
                    continue
                
    return(previous_5)
                    



In [41]:
#a function to rename those teams whose players have same last names

def rename_same_names(team, s5_team, NBA_database):
    players_ln = NBA_dataset[NBA_dataset['team'] == team]['last_name'].values.tolist()
    players_fn = NBA_dataset[NBA_dataset['team'] == team]['first_name'].values.tolist()
    new_s5_team = []
    
    ln_player_count = get_player_count(players_ln)
    
    for name in s5_team:
        if ln_player_count[name] == 1:
            new_s5_team.append(name)
        else:
            
            for i in range(0, len(players_ln)):
                
                if players_ln[i] == name:
                    
                    first_name_abbrev = players_fn[i][0]
                    
                    fna_plus_name = first_name_abbrev + " " + name                    
                    if fna_plus_name not in new_s5_team:
                        new_s5_team.append(fna_plus_name)
    
    
    return (new_s5_team)
    

In [42]:
# for the position dataset: we want to convert names to positions

def get_names_and_pos(team, s5_names, NBA_dataset):
    
    pos5_team = []
    rn5_team = []
    
    relevant_data = NBA_dataset[NBA_dataset['team'] == team]
    
    

    for names in s5_names:
        curr_length_team = len(pos5_team)
        
        #case 1: normal case, just one name
        if len(names.split(" ")) == 1: 
            
            rel = relevant_data.loc[relevant_data['last_name'] == names]
            
            #simple case
            if rel.empty == False:
                for i, r in rel.iterrows():                    
                    pos5_team.append(r['pos'])
                    rn5_team.append(r['names'])
                    break
            
            #name is McAdoo; the one recorded on the dataset is Michael McAdoo
            else:
                #mcadoo case
                rel2 = NBA_dataset.loc[NBA_dataset['team'] == team]

                for i,j in rel2.iterrows():
                    if names in j['last_name']:
                        pos5_team.append(j['pos'])
                        rn5_team.append(j['names'])
            
            

        
        else:
            names = names.replace(".","")
            full_name = names.split(" ")
            fn = full_name.pop(0)
            last_name = " ".join(full_name)
            
            rel2 = relevant_data.loc[relevant_data['last_name'] == last_name]
            
            # michael mcadoo or smth
            if rel2.empty:
                rel2 = relevant_data.loc[relevant_data['last_name'] == names]
            
            #nance jr
            if rel2.empty:
                rel2 = relevant_data.loc[relevant_data['last_name'] == fn]

            for index, row in rel2.iterrows():
                
                if last_name in row['last_name'] and fn in row['first_name'].replace(".",""):
                    pos5_team.append(row['pos'])
                    rn5_team.append(row['names'])
                    break
                    
                #k. thompson; klay thompson
                elif fn in row['first_name']:
                    pos5_team.append(row['pos'])
                    rn5_team.append(row['names'])
                    break
                
                elif fn in row['last_name']:
                    pos5_team.append(row['pos'])
                    rn5_team.append(row['names'])
                    break
                 
                
        
        #if same pa rin, then do the super last resort, which is lcs
        if curr_length_team == len(pos5_team) :

            #super last resort: get the full name, then get the longest common substring in the whole team

            lcs_word = ''
            lcs_index = 0
            for i2,row in relevant_data.iterrows():
                longest_sub = lcs(names.encode("utf-8"), str(row['names']))
                longest_sub = "".join(list(longest_sub))
                if len(longest_sub) > len(lcs_word):
                    lcs_word = longest_sub
                    lcs_index = i2

            #now, we go to the said index and record the data
            pos5_team.append(NBA_dataset.iloc[lcs_index]['pos'])
            rn5_team.append(NBA_dataset.iloc[lcs_index]['names'])

        

    return [pos5_team, rn5_team]

#reval = convert_names_to_pos("GSW", s5_GSW_bns, NBA_dataset)
    

    

In [43]:
def lcs(S,T):
    m = len(S)
    n = len(T)
    counter = [[0]*(n+1) for x in range(m+1)]
    longest = 0
    lcs_set = set()
    for i in range(m):
        for j in range(n):
            if S[i] == T[j]:
                c = counter[i][j] + 1
                counter[i+1][j+1] = c
                if c > longest:
                    lcs_set = set()
                    longest = c
                    lcs_set.add(S[i-c+1:i+1])
                elif c == longest:
                    lcs_set.add(S[i-c+1:i+1])

    return lcs_set



# MAIN


### Read the NBA File.

In [44]:
NBA_dataset = pd.read_csv("NBA Dataset with RPM.csv", sep=",")
NBA_dataset.head()

,first_name,last_name,names,pos,team,GP,MPG,ORPM,DRPM,RPM,WINS
0,Giannis,Antetokounmpo,Giannis Antetokounmpo,PG,MIL,80.0,35.3,0.84,0.27,1.11,7.46
1,Jerryd,Bayless,Jerryd Bayless,SG,MIL,52.0,28.9,0.21,-2.34,-2.13,0.92
2,Michael,Carter-Williams,Michael Carter-Williams,PG,MIL,54.0,30.5,-1.73,0.37,-1.36,1.82
3,Chris,Copeland,Chris Copeland,PF,MIL,NaN,NaN,NaN,NaN,NaN,NaN
4,Jared,Cunningham,Jared Cunningham,SG,MIL,NaN,NaN,NaN,NaN,NaN,NaN


## Scrape the list of urls of play-by-plays using the Wikipedia url.

In [45]:
#get all the URLs for scraping!
#url composition: "http://www.nba.com/games/" + <date of game from wiki> + "//" + <team A AT team B>  + "//" + "/gameinfo.html"

#get url 
wiki_url = "https://en.wikipedia.org/wiki/2015–16_Golden_State_Warriors_season"
url = wiki_url

#use beautiful soup to parse more conveniently
r1 = urllib2.urlopen(url)
soup = BeautifulSoup(r1, 'html.parser')

wiki_table = soup.findAll('table', {'class':'toccolours'})

#get the relevant tables; i.e. the regular season sched and playoffs sched. We need these tables to generate the links
reg_season_sched = wiki_table[1]
playoffs_sched = wiki_table[2]



In [46]:
#regular season table
#get the date and the team acronyms for visiting the NBA play-by-play score
games_log = str(reg_season_sched).split('<tr style="background')
games_log = games_log[1:]

dates_teams = []
dates_teams_2 = []
for i in games_log:
    temp = i.split("https://watch.nba.com/game/")
    date_player = temp[1].split('"')
    dates_teams.append(date_player[0])


In [47]:
#playoffs season table

#comment this entire cell if you don't want playoff values
#get the date and the team acronyms for visiting the NBA play-by-play score
games_log = str(playoffs_sched).split('<tr style="background')

games_log = games_log[1:]

for i in games_log:
    temp = i.split("https://watch.nba.com/game/")
    date_player = temp[1].split('"')
    dates_teams_2.append(date_player[0])


In [48]:
#concatenate links from both regular season and playoffs
dates_teams = dates_teams + dates_teams_2
print(len(dates_teams))
#print dates_teams_2

106


## Iterate through the dates_teams list to scrape the play-by-play games of GSW.
### Each line is a game before the nest substitution happened.

In [49]:
date_team = dates_teams[:]
print len(date_team)


106


In [72]:
# Iterate through all dates_teams, record all that errored
print str(datetime.now())
may_topak = []
indexes = []
error_message = []

no_topak_indexes = []

pos5_GSW_bns = []
pos5_OPP_bns = []

pbp_data = pd.DataFrame(columns=('id','p1nG', 'p2nG', 'p3nG', 'p4nG', 'p5nG', 'p1nO', 'p2nO','p3nO','p4nO','p5nO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))
pos_pbp_data = pd.DataFrame(columns=('id','p1pG', 'p2pG', 'p3pG', 'p4pG', 'p5pG', 'p1pO', 'p2pO','p3pO','p4pO','p5pO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))
                
ultra_pbp_data = pd.DataFrame(columns=('id','p1nG', 'p2nG', 'p3nG', 'p4nG', 'p5nG', 'p1nO', 'p2nO','p3nO','p4nO','p5nO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))
pos_ultra_pbp_data = pd.DataFrame(columns=('id','p1pG', 'p2pG', 'p3pG', 'p4pG', 'p5pG', 'p1pO', 'p2pO','p3pO','p4pO','p5pO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))

marker = 0
for j in range(0, len(date_team)):
    
    try:
        pbp_data = pd.DataFrame(columns=('id','p1nG', 'p2nG', 'p3nG', 'p4nG', 'p5nG', 'p1nO', 'p2nO','p3nO','p4nO','p5nO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))
        pos_pbp_data = pd.DataFrame(columns=('id','p1pG', 'p2pG', 'p3pG', 'p4pG', 'p5pG', 'p1pO', 'p2pO','p3pO','p4pO','p5pO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))

        #visit the url with the play-by-play
        samp_url = "http://www.nba.com/games/" + date_team[j] + "/gameinfo.html"
        r1 = urllib2.urlopen(samp_url)
        soup = BeautifulSoup(r1, 'html.parser')

        
        #determine which team is the left; which is right
        teams = date_team[j].split("/")[1]
        val = teams.index("GSW")
        other_team = teams.replace('GSW','')

        
        #consider Brooklyn, Charlotte, and Phoenix cases; 
        # they are abbreviated differently at basketball-references.com
        if other_team == 'BKN':
            other_team = 'BRK'
        elif other_team == 'CHA':
            other_team = 'CHO'
        elif other_team == 'PHX':
            other_team = 'PHO'

        if (val == 0):
            left_team = "GSW"
            right_team = other_team
        else:
            left_team = other_team
            right_team = "GSW"
        

        #print "LEFT TEAM: ", left_team
        #print "RIGHT TEAM: ", right_team
        
        #print(samp_url)
        
        
        # get the starting 10 people 
        starting_10 = soup.findAll('td',{'class':'b'})

        s5_GSW_bns = []
        s5_OPP_bns = []

        for i in range(0, len(starting_10)):

            player_name = starting_10[i].text

            #if left team; else right team
            if (i <= 4):
                if (left_team == "GSW"):
                    s5_GSW_bns.append(player_name)
                else:
                    s5_OPP_bns.append(player_name)
            else:
                if (right_team == "GSW"):
                    s5_GSW_bns.append(player_name)
                else:
                    s5_OPP_bns.append(player_name)

        #print("GSW Starting 5:", s5_GSW_bns)
        #print("OPP Starting 5:", s5_OPP_bns)

        
        # get the play-by-play columns
        left_pbp_col = soup.findAll('td',{'class':{'nbaGIPbPLft','nbaGIPbPLftScore'}})
        mid_pbp_col = soup.findAll('td',{'class':{'nbaGIPbPMid','nbaGIPbPMidScore'}})
        right_pbp_col = soup.findAll('td',{'class':{'nbaGIPbPRgt','nbaGIPbPRgtScore'}})

        assert len(left_pbp_col) == len(mid_pbp_col)
        assert len(mid_pbp_col) == len(right_pbp_col)
        
        
        #where blood starts to show. 
        #essentially, we want to record every game detail
        #each row contains details before the next substitution
        
        scores = {'GSW': 0, 'OPP': 0}
        scores_bns = {'GSW': 0, 'OPP': 0}

        time = {'min':0, 'sec':0}
        time_bns = {'min': 12, 'sec':0}

        quarter = 1

        #iterate through the entire list of pbp data
        for i in range(0, len(left_pbp_col)):

            #if there's a subsitution, record the values to pbp_data, then make the changes
            if "Substitution" in left_pbp_col[i].text or "Substitution" in right_pbp_col[i].text:

                #print("\n\n===SUBSTITUTION===")

                assert len(set(s5_GSW_bns)) == 5
                assert len(set(s5_OPP_bns)) == 5
                
                #get the difference of time 
                time_diff = subtract_time(time, time_bns)
                time_diff_secs = 60*time_diff['min'] + time_diff['sec']
                time_bns = time.copy()

                #get the difference of score
                score_diff_GSW = scores['GSW'] - scores_bns['GSW']
                score_diff_OPP = scores['OPP'] - scores_bns['OPP']
                sO_minus_sG = score_diff_OPP - score_diff_GSW
                scores_bns = scores.copy()

                #record the players, and the mins and score            
                #record = [i] + s5_GSW_bns + s5_OPP_bns + [time_diff_secs, score_diff_GSW, score_diff_OPP, sG_minus_sO]

                ### before recording, convert the names to positions
                #print "Part of the code where we convert names to positions."
                pos5_GSW_bns,s5_GSW_bns_rn = get_names_and_pos("GSW", s5_GSW_bns[:], NBA_dataset)
                pos5_OPP_bns,s5_OPP_bns_rn = get_names_and_pos(other_team, s5_OPP_bns[:], NBA_dataset)
                ###
                
                pos_record = [marker] + pos5_GSW_bns + pos5_OPP_bns + [time_diff_secs, score_diff_GSW, score_diff_OPP, sO_minus_sG] + [date_team[j]]
                record = [marker] + s5_GSW_bns_rn + s5_OPP_bns_rn + [time_diff_secs, score_diff_GSW, score_diff_OPP, sO_minus_sG]  + [date_team[j]]
                marker += 1
                
                assert len(set(s5_GSW_bns)) == 5
                assert len(set(s5_OPP_bns)) == 5
                

                #actually recording the data in their respective data frames
                temp_df = pd.DataFrame([record],columns=('id','p1nG', 'p2nG', 'p3nG', 'p4nG', 'p5nG', 'p1nO', 'p2nO','p3nO','p4nO','p5nO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG', 'game'))
                frames = [pbp_data, temp_df]
                pbp_data = pd.concat(frames, ignore_index = True)

                pos_temp_df = pd.DataFrame([pos_record], columns=('id','p1pG', 'p2pG', 'p3pG', 'p4pG', 'p5pG', 'p1pO', 'p2pO','p3pO','p4pO','p5pO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))
                pos_frames = [pos_pbp_data, pos_temp_df]
                pos_pbp_data = pd.concat(pos_frames, ignore_index = True)
                

                
                #now we've recorded the data, make the substitution
                left_text = left_pbp_col[i].text
                right_text = right_pbp_col[i].text

                if len(left_text) > len(right_text):
                    relevant_text = left_text
                    replacing_team = left_team
                else:
                    relevant_text = right_text
                    replacing_team = right_team

                #print ("Text of Sub:", relevant_text)
                #print ("Replacing Team: ", replacing_team)

                #change team players, and consider the Jones/Jones case 
                if (replacing_team == 'GSW'):
                    s5_GSW_bns = replace_players(s5_GSW_bns, relevant_text)
                    
                    #if there are same surnames in the list
                    if len(set(s5_GSW_bns)) != 5:
                        s5_GSW_bns = rename_same_names(replacing_team, s5_GSW_bns, NBA_dataset)
                        
                elif (replacing_team == other_team):
                    s5_OPP_bns = replace_players(s5_OPP_bns, relevant_text)
                        
                    if len(set(s5_OPP_bns)) != 5:
                        s5_OPP_bns = rename_same_names(replacing_team, s5_OPP_bns, NBA_dataset)
                
                
                #print ("Teams After Sub:")
                #print("GSW:", s5_GSW_bns)
                #print("OPP:", s5_OPP_bns)
                
                assert len(set(s5_GSW_bns)) == 5
                assert len(set(s5_OPP_bns)) == 5
                #print("===RETURN TO GAME===\n\n")

            # even if there is no substition, we should always keep track of the score and time

            # update score and time    
            time_score = mid_pbp_col[i].text.split(' [')
            if len(time_score) > 1:
                update_scores(time_score[1], scores)

            update_time(time_score[0], time)

            #reset time_bns to 12:00 if new quarter was seen
            if subtract_time(time, time_bns)['min'] < 0:
                time_bns = {'min': 12, 'sec':0}
            
            
                #print ("New Quarter has Arrived. Teams before this next quarter (GSW / OPP): ")
                #print s5_GSW_bns
                #print s5_OPP_bns
                #print ("\n\n We predict the next starting 5 because NBA coaches are inconsistent and they do not follow the last five people.")

                quarter += 1
                #print ("Quarter Number: ", quarter)

                #print ("Index Number: ", i)
                if (left_team == "GSW"):
                    s5_GSW_bns = predict_new_starting_5(left_team, s5_GSW_bns, left_pbp_col, NBA_dataset, i)
                    s5_OPP_bns = predict_new_starting_5(right_team, s5_OPP_bns, right_pbp_col, NBA_dataset, i)
                elif (right_team == "GSW"):
                    s5_GSW_bns = predict_new_starting_5(right_team, s5_GSW_bns, right_pbp_col, NBA_dataset, i)
                    s5_OPP_bns = predict_new_starting_5(left_team, s5_OPP_bns, left_pbp_col, NBA_dataset, i)


                #print ("This is the end of the prediction. I am still very sad about the fact that I can't just use the last quarter.")

        #end of iterate through the entire list
        #at this point, we finished recording the play-by-play of 1 game
        
        #record one last time for the remaining time

        
        #print("\n\n===LAST RECORDING===")

        time_diff = subtract_time(time, time_bns)
        time_diff_secs = 60*time_diff['min'] + time_diff['sec']
        time_bns = time.copy()
        score_diff_GSW = scores['GSW'] - scores_bns['GSW']
        score_diff_OPP = scores['OPP'] - scores_bns['OPP']
        sO_minus_sG = score_diff_OPP - score_diff_GSW
        scores_bns = scores.copy()
        ### before recording, convert the names to positions
        pos5_GSW_bns,s5_GSW_bns_rn = get_names_and_pos("GSW", s5_GSW_bns, NBA_dataset)
        pos5_OPP_bns,s5_OPP_bns_rn = get_names_and_pos(other_team, s5_OPP_bns, NBA_dataset)
        ###
        pos_record2 = [marker] + pos5_GSW_bns + pos5_OPP_bns + [time_diff_secs, score_diff_GSW, score_diff_OPP, sO_minus_sG]  + [date_team[j]]
        record2 = [marker] + s5_GSW_bns_rn + s5_OPP_bns_rn + [time_diff_secs, score_diff_GSW, score_diff_OPP, sO_minus_sG]  + [date_team[j]]
        marker += 1
        assert len(set(s5_GSW_bns)) == 5
        assert len(set(s5_OPP_bns)) == 5
        temp_df = pd.DataFrame([record2],columns=('id','p1nG', 'p2nG', 'p3nG', 'p4nG', 'p5nG', 'p1nO', 'p2nO','p3nO','p4nO','p5nO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))
        frames = [pbp_data, temp_df]
        pbp_data = pd.concat(frames, ignore_index = True)
        pos_temp_df = pd.DataFrame([pos_record2], columns=('id','p1pG', 'p2pG', 'p3pG', 'p4pG', 'p5pG', 'p1pO', 'p2pO','p3pO','p4pO','p5pO', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))
        pos_frames = [pos_pbp_data, pos_temp_df]
        pos_pbp_data = pd.concat(pos_frames, ignore_index = True)
        #print("===END OF GAME===\n")
        
        #here we collate all the data we got into one big dataset

        frames2 = [pbp_data, ultra_pbp_data]
        ultra_pbp_data = pd.concat(frames2, ignore_index = True)
        
        pos_frames2 = [pos_pbp_data, pos_ultra_pbp_data]
        pos_ultra_pbp_data = pd.concat(pos_frames2, ignore_index = True)
        
        no_topak_indexes.append(j)
        
        print "finished with #: ", str(j), " ", date_team[j]
        
        
    except Exception as e:
        print "Skipping ", date_team[j], " ..."
        print e
        may_topak.append(date_team[j])
        error_message.append(e)
        indexes.append(j)
print str(datetime.now())

2018-03-08 15:33:42.986000
finished with #:  0   20151027/NOPGSW
finished with #:  1   20151030/GSWHOU
finished with #:  2   20151031/GSWNOP
finished with #:  3   20151102/MEMGSW
finished with #:  4   20151104/LACGSW
finished with #:  5   20151106/DENGSW
finished with #:  6   20151107/GSWSAC
finished with #:  7   20151109/DETGSW
finished with #:  8   20151111/GSWMEM
finished with #:  9   20151112/GSWMIN
finished with #:  10   20151114/BKNGSW
finished with #:  11   20151117/TORGSW
finished with #:  12   20151119/GSWLAC
finished with #:  13   20151120/CHIGSW
finished with #:  14   20151122/GSWDEN
finished with #:  15   20151124/LALGSW
finished with #:  16   20151127/GSWPHX
finished with #:  17   20151128/SACGSW
finished with #:  18   20151130/GSWUTA
finished with #:  19   20151202/GSWCHA
finished with #:  20   20151205/GSWTOR
finished with #:  21   20151206/GSWBKN
finished with #:  22   20151208/GSWIND
finished with #:  23   20151211/GSWBOS
finished with #:  24   20151212/GSWMIL
finished

In [73]:
print "Errors:"
print may_topak
print indexes
print error_message
len(may_topak)
print "---"
print "Number of Properly Recorded Games:"
print len(no_topak_indexes)

Errors:
[]
[]
[]
---
Number of Properly Recorded Games:
106


In [74]:
ultra_pbp_data.to_csv("Pbp Data Names.csv", sep=',',index=False)


In [75]:
pos_ultra_pbp_data.to_csv("Pbp Data Positions.csv", sep = ',',index=False)

In [77]:
ultra_pbp_data.head(51)

,id,p1nG,p2nG,p3nG,p4nG,p5nG,p1nO,p2nO,p3nO,p4nO,p5nO,secs,score_GSW,score_OPP,sO-sG,game
0,4637,Harrison Barnes,Draymond Green,Festus Ezeli,Klay Thompson,Stephen Curry,LeBron James,Kevin Love,Tristan Thompson,J.R. Smith,Kyrie Irving,302,8,8,0,20160619/CLEGSW
1,4638,Harrison Barnes,Draymond Green,Andre Iguodala,Klay Thompson,Stephen Curry,LeBron James,Kevin Love,Tristan Thompson,J.R. Smith,Kyrie Irving,84,3,2,-1,20160619/CLEGSW
2,4639,Harrison Barnes,Draymond Green,Andre Iguodala,Klay Thompson,Stephen Curry,LeBron James,Richard Jefferson,Tristan Thompson,J.R. Smith,Kyrie Irving,185,5,6,1,20160619/CLEGSW
3,4640,Anderson Varejao,Draymond Green,Andre Iguodala,Klay Thompson,Stephen Curry,LeBron James,Richard Jefferson,Tristan Thompson,J.R. Smith,Kyrie Irving,0,0,0,0,20160619/CLEGSW
4,4641,Anderson Varejao,Draymond Green,Andre Iguodala,Shaun Livingston,Stephen Curry,LeBron James,Richard Jefferson,Tristan Thompson,J.R. Smith,Kyrie Irving,0,0,0,0,20160619/CLEGSW
5,4642,Anderson Varejao,Draymond Green,Andre Iguodala,Shaun Livingston,Stephen Curry,LeBron James,Richard Jefferson,Kevin Love,J.R. Smith,Kyrie Irving,53,3,2,-1,20160619/CLEGSW
6,4643,Anderson Varejao,Draymond Green,Andre Iguodala,Shaun Livingston,Stephen Curry,Iman Shumpert,Richard Jefferson,Kevin Love,J.R. Smith,Kyrie Irving,30,2,1,-1,20160619/CLEGSW
7,4644,Anderson Varejao,Harrison Barnes,Andre Iguodala,Shaun Livingston,Stephen Curry,Iman Shumpert,Richard Jefferson,Kevin Love,J.R. Smith,Kyrie Irving,41,1,2,1,20160619/CLEGSW
8,4645,Anderson Varejao,Harrison Barnes,Andre Iguodala,Shaun Livingston,Leandro Barbosa,Iman Shumpert,Richard Jefferson,Kevin Love,J.R. Smith,Kyrie Irving,0,0,0,0,20160619/CLEGSW
9,4646,Anderson Varejao,Harrison Barnes,Andre Iguodala,Shaun Livingston,Leandro Barbosa,LeBron James,Richard Jefferson,Kevin Love,J.R. Smith,Kyrie Irving,21,0,1,1,20160619/CLEGSW


In [78]:
pos_ultra_pbp_data.head()

,id,p1pG,p2pG,p3pG,p4pG,p5pG,p1pO,p2pO,p3pO,p4pO,p5pO,secs,score_GSW,score_OPP,sO-sG,game
0,4637,SF,PF,C,SG,PG,SF,PF,PF,SG,PG,302,8,8,0,20160619/CLEGSW
1,4638,SF,PF,SF,SG,PG,SF,PF,PF,SG,PG,84,3,2,-1,20160619/CLEGSW
2,4639,SF,PF,SF,SG,PG,SF,SF,PF,SG,PG,185,5,6,1,20160619/CLEGSW
3,4640,C,PF,SF,SG,PG,SF,SF,PF,SG,PG,0,0,0,0,20160619/CLEGSW
4,4641,C,PF,SF,PG,PG,SF,SF,PF,SG,PG,0,0,0,0,20160619/CLEGSW
